# 🛠️ Training BDH for Tool Calling

Train a specialized BDH model for function/tool calling that can be composed with other models.

## Goal

Create a model that can:
1. Recognize when a tool call is needed
2. Generate valid JSON function calls
3. Parse tool definitions and match to queries

## Target Format (pydantic-ai compatible)

```
User: What's the weather in Tokyo?
Tools: [{"name": "get_weather", "params": {"location": "string"}}]
Assistant: {"name": "get_weather", "arguments": {"location": "Tokyo"}}
```

## Hardware
- A100 80GB recommended for Large config
- ~40GB VRAM expected usage

In [ ]:
# Setup
!pip install torch datasets transformers tqdm matplotlib -q

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import json
import os
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name()}')
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

In [ ]:
# Clone BDH repo if needed
if not os.path.exists('hierarchical_bdh.py'):
    !git clone https://github.com/newsbubbles/bdh.git temp_bdh
    !cp temp_bdh/bdh.py .
    !cp temp_bdh/hierarchical_bdh.py .
    !rm -rf temp_bdh

from hierarchical_bdh import HierarchicalBDH, HierarchicalBDHConfig
print('BDH loaded!')

---
## Step 1: Load Tool-Calling Datasets

We'll combine multiple high-quality function-calling datasets.

In [ ]:
from datasets import load_dataset

# Load Glaive function calling dataset (high quality)
print('Loading Glaive function-calling dataset...')
try:
    glaive = load_dataset('glaiveai/glaive-function-calling-v2', split='train')
    print(f'Glaive: {len(glaive)} examples')
except Exception as e:
    print(f'Could not load Glaive: {e}')
    glaive = None

# Load Gorilla OpenFunctions
print('Loading Gorilla OpenFunctions...')
try:
    gorilla = load_dataset('gorilla-llm/Berkeley-Function-Calling-Leaderboard', split='train')
    print(f'Gorilla: {len(gorilla)} examples')
except Exception as e:
    print(f'Could not load Gorilla: {e}')
    gorilla = None

# Load NousResearch Hermes function calling
print('Loading Hermes function-calling...')
try:
    hermes = load_dataset('NousResearch/hermes-function-calling-v1', split='train')
    print(f'Hermes: {len(hermes)} examples')
except Exception as e:
    print(f'Could not load Hermes: {e}')
    hermes = None

In [ ]:
# Examine dataset formats
print('='*60)
print('DATASET FORMAT INSPECTION')
print('='*60)

if glaive:
    print('\nGlaive sample:')
    print(glaive[0])

if gorilla:
    print('\nGorilla sample:')
    print(gorilla[0])

if hermes:
    print('\nHermes sample:')
    print(hermes[0])

In [ ]:
# Standardize format for training
# Target format:
# <|system|>You are a helpful assistant with access to tools.
# <|tools|>[{"name": "func", "description": "...", "parameters": {...}}]
# <|user|>User query here
# <|assistant|>{"name": "func", "arguments": {...}}

def format_glaive_example(example):
    """Convert Glaive format to our standard format."""
    try:
        # Glaive has 'system', 'chat' fields
        system = example.get('system', '')
        chat = example.get('chat', '')
        
        # Extract tools from system prompt
        tools_str = ''
        if 'functions' in system.lower():
            # Try to extract JSON
            import re
            match = re.search(r'\[.*?\]', system, re.DOTALL)
            if match:
                tools_str = match.group(0)
        
        formatted = f"<|system|>You are a helpful assistant with tool access.\n"
        if tools_str:
            formatted += f"<|tools|>{tools_str}\n"
        formatted += chat
        
        return formatted
    except:
        return None

def format_hermes_example(example):
    """Convert Hermes format to our standard format."""
    try:
        conversations = example.get('conversations', [])
        tools = example.get('tools', [])
        
        formatted = "<|system|>You are a helpful assistant with tool access.\n"
        if tools:
            formatted += f"<|tools|>{json.dumps(tools)}\n"
        
        for conv in conversations:
            role = conv.get('from', conv.get('role', ''))
            content = conv.get('value', conv.get('content', ''))
            
            if role in ['human', 'user']:
                formatted += f"<|user|>{content}\n"
            elif role in ['gpt', 'assistant']:
                formatted += f"<|assistant|>{content}\n"
        
        return formatted
    except:
        return None

In [ ]:
# Process and combine datasets
all_examples = []

if glaive:
    print('Processing Glaive...')
    for ex in tqdm(glaive):
        formatted = format_glaive_example(ex)
        if formatted and len(formatted) > 100:
            all_examples.append(formatted)
    print(f'Added {len(all_examples)} from Glaive')

prev_count = len(all_examples)
if hermes:
    print('Processing Hermes...')
    for ex in tqdm(hermes):
        formatted = format_hermes_example(ex)
        if formatted and len(formatted) > 100:
            all_examples.append(formatted)
    print(f'Added {len(all_examples) - prev_count} from Hermes')

print(f'\nTotal examples: {len(all_examples)}')

In [ ]:
# If datasets didn't load, create synthetic examples
if len(all_examples) < 1000:
    print('Creating synthetic tool-calling examples...')
    
    # Define some tools
    tools = [
        {
            "name": "get_weather",
            "description": "Get current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"},
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
                },
                "required": ["location"]
            }
        },
        {
            "name": "search_web",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"},
                    "num_results": {"type": "integer", "default": 5}
                },
                "required": ["query"]
            }
        },
        {
            "name": "send_email",
            "description": "Send an email to a recipient",
            "parameters": {
                "type": "object",
                "properties": {
                    "to": {"type": "string", "description": "Recipient email"},
                    "subject": {"type": "string", "description": "Email subject"},
                    "body": {"type": "string", "description": "Email body"}
                },
                "required": ["to", "subject", "body"]
            }
        },
        {
            "name": "calculate",
            "description": "Perform a mathematical calculation",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Math expression"}
                },
                "required": ["expression"]
            }
        },
        {
            "name": "get_stock_price",
            "description": "Get current stock price",
            "parameters": {
                "type": "object",
                "properties": {
                    "symbol": {"type": "string", "description": "Stock ticker symbol"}
                },
                "required": ["symbol"]
            }
        },
        {
            "name": "create_reminder",
            "description": "Create a reminder",
            "parameters": {
                "type": "object",
                "properties": {
                    "message": {"type": "string", "description": "Reminder message"},
                    "time": {"type": "string", "description": "When to remind"}
                },
                "required": ["message", "time"]
            }
        },
        {
            "name": "translate",
            "description": "Translate text between languages",
            "parameters": {
                "type": "object",
                "properties": {
                    "text": {"type": "string", "description": "Text to translate"},
                    "source_lang": {"type": "string", "description": "Source language"},
                    "target_lang": {"type": "string", "description": "Target language"}
                },
                "required": ["text", "target_lang"]
            }
        },
        {
            "name": "read_file",
            "description": "Read contents of a file",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {"type": "string", "description": "File path"}
                },
                "required": ["path"]
            }
        },
        {
            "name": "write_file",
            "description": "Write content to a file",
            "parameters": {
                "type": "object",
                "properties": {
                    "path": {"type": "string", "description": "File path"},
                    "content": {"type": "string", "description": "Content to write"}
                },
                "required": ["path", "content"]
            }
        },
        {
            "name": "run_python",
            "description": "Execute Python code",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {"type": "string", "description": "Python code to execute"}
                },
                "required": ["code"]
            }
        }
    ]
    
    # Example queries and responses for each tool
    examples_per_tool = {
        "get_weather": [
            ("What's the weather in Tokyo?", {"location": "Tokyo"}),
            ("How's the weather in New York today?", {"location": "New York"}),
            ("Is it cold in London?", {"location": "London"}),
            ("Weather forecast for Paris please", {"location": "Paris"}),
            ("What's the temperature in celsius in Berlin?", {"location": "Berlin", "unit": "celsius"}),
            ("Give me weather in fahrenheit for Miami", {"location": "Miami", "unit": "fahrenheit"}),
        ],
        "search_web": [
            ("Search for Python tutorials", {"query": "Python tutorials"}),
            ("Find information about machine learning", {"query": "machine learning"}),
            ("Look up the latest news on AI", {"query": "latest AI news"}),
            ("Search for best restaurants nearby", {"query": "best restaurants nearby"}),
            ("Find 10 results about climate change", {"query": "climate change", "num_results": 10}),
        ],
        "send_email": [
            ("Send an email to john@example.com about the meeting tomorrow", {"to": "john@example.com", "subject": "Meeting Tomorrow", "body": "Hi John, just a reminder about our meeting tomorrow."}),
            ("Email sarah@company.com the project update", {"to": "sarah@company.com", "subject": "Project Update", "body": "Here is the latest project update."}),
        ],
        "calculate": [
            ("What's 25 times 47?", {"expression": "25 * 47"}),
            ("Calculate 15% of 200", {"expression": "0.15 * 200"}),
            ("What's the square root of 144?", {"expression": "sqrt(144)"}),
            ("Compute 2 to the power of 10", {"expression": "2 ** 10"}),
        ],
        "get_stock_price": [
            ("What's Apple's stock price?", {"symbol": "AAPL"}),
            ("Get me the price of TSLA", {"symbol": "TSLA"}),
            ("How is Microsoft stock doing?", {"symbol": "MSFT"}),
            ("Check NVDA stock", {"symbol": "NVDA"}),
        ],
        "create_reminder": [
            ("Remind me to call mom at 5pm", {"message": "Call mom", "time": "5pm"}),
            ("Set a reminder for the meeting in 2 hours", {"message": "Meeting", "time": "in 2 hours"}),
            ("Remind me to take medication tomorrow morning", {"message": "Take medication", "time": "tomorrow morning"}),
        ],
        "translate": [
            ("Translate 'Hello world' to Spanish", {"text": "Hello world", "target_lang": "Spanish"}),
            ("How do you say 'thank you' in French?", {"text": "thank you", "target_lang": "French"}),
            ("Translate this to German: Good morning", {"text": "Good morning", "target_lang": "German"}),
        ],
        "read_file": [
            ("Read the contents of config.json", {"path": "config.json"}),
            ("Show me what's in /home/user/notes.txt", {"path": "/home/user/notes.txt"}),
            ("Open and read data.csv", {"path": "data.csv"}),
        ],
        "write_file": [
            ("Write 'Hello World' to output.txt", {"path": "output.txt", "content": "Hello World"}),
            ("Save this note to notes.md: Remember to review PR", {"path": "notes.md", "content": "Remember to review PR"}),
        ],
        "run_python": [
            ("Run this Python code: print('Hello')", {"code": "print('Hello')"}),
            ("Execute: for i in range(5): print(i)", {"code": "for i in range(5): print(i)"}),
            ("Run a Python script to list files", {"code": "import os; print(os.listdir('.'))"}),
        ],
    }
    
    # Generate training examples
    import random
    
    for _ in range(10000):  # Generate 10K examples
        # Pick 1-4 random tools to make available
        num_tools = random.randint(1, 4)
        available_tools = random.sample(tools, num_tools)
        
        # Pick one tool to actually use
        tool = random.choice(available_tools)
        tool_name = tool['name']
        
        if tool_name in examples_per_tool:
            query, args = random.choice(examples_per_tool[tool_name])
            
            # Format the example
            example = f"<|system|>You are a helpful assistant with tool access.\n"
            example += f"<|tools|>{json.dumps(available_tools)}\n"
            example += f"<|user|>{query}\n"
            example += f"<|assistant|>{json.dumps({'name': tool_name, 'arguments': args})}\n"
            
            all_examples.append(example)
    
    print(f'Generated {len(all_examples)} synthetic examples')

In [ ]:
# Show some examples
print('Sample training examples:')
print('='*60)
for i in range(3):
    print(all_examples[i][:500])
    print('-'*60)

---
## Step 2: Create Dataset and DataLoader

In [ ]:
class ToolCallingDataset(Dataset):
    def __init__(self, examples, block_size=1024):
        self.examples = examples
        self.block_size = block_size
        
        # Concatenate all examples into one big byte sequence
        self.data = torch.tensor(
            [b for ex in examples for b in ex.encode('utf-8')],
            dtype=torch.long
        )
        print(f'Dataset size: {len(self.data):,} bytes')
        
    def __len__(self):
        return max(1, len(self.data) - self.block_size - 1)
    
    def __getitem__(self, idx):
        x = self.data[idx:idx + self.block_size]
        y = self.data[idx + 1:idx + self.block_size + 1]
        return x, y

# Create dataset
block_size = 1024  # Longer context for tool definitions
dataset = ToolCallingDataset(all_examples, block_size=block_size)

# Split into train/val
train_size = int(0.95 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

print(f'Train: {len(train_dataset)}, Val: {len(val_dataset)}')

---
## Step 3: Configure Large Model

Using Large config for better tool-calling capability.

In [ ]:
# Large config - should fit in ~40GB VRAM on A100
config = HierarchicalBDHConfig(
    vocab_size=256,
    n_embd=512,
    local_n_layer=6,
    global_n_layer=6,
    n_head=8,
    patch_size=32,  # Larger patches for longer context
    mlp_internal_dim_multiplier=128,
    dropout=0.1,
)

model = HierarchicalBDH(config).to(device)

# Count parameters
n_params = sum(p.numel() for p in model.parameters())
print(f'Model parameters: {n_params:,}')
print(f'Estimated VRAM: ~{n_params * 4 / 1e9 * 3:.1f} GB')  # params + grads + optimizer

In [ ]:
# Training config
batch_size = 32
learning_rate = 3e-4
warmup_steps = 500
max_steps = 10000
eval_interval = 500
save_interval = 1000
gradient_accumulation_steps = 2  # Effective batch = 64

# Optimizer
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    betas=(0.9, 0.95),
    weight_decay=0.1
)

# Learning rate scheduler with warmup
def get_lr(step):
    if step < warmup_steps:
        return learning_rate * step / warmup_steps
    # Cosine decay
    decay_ratio = (step - warmup_steps) / (max_steps - warmup_steps)
    return learning_rate * 0.1 + 0.5 * (learning_rate - learning_rate * 0.1) * (1 + np.cos(np.pi * decay_ratio))

print('Training config ready!')

---
## Step 4: Training Loop

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Training state
train_losses = []
val_losses = []
best_val_loss = float('inf')

# Checkpoint directory
os.makedirs('checkpoints_tool_calling', exist_ok=True)

print('Starting training...')

In [ ]:
@torch.no_grad()
def evaluate():
    model.eval()
    losses = []
    for x, y in val_loader:
        x, y = x.to(device), y.to(device)
        
        # Ensure divisible by patch_size
        valid_len = (x.size(1) // config.patch_size) * config.patch_size
        if valid_len < config.patch_size:
            continue
        x = x[:, :valid_len]
        y = y[:, :valid_len]
        
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, 256), y.view(-1))
        losses.append(loss.item())
    
    model.train()
    return np.mean(losses) if losses else float('inf')

In [ ]:
# Main training loop
model.train()
step = 0
accum_loss = 0

pbar = tqdm(total=max_steps, desc='Training')

while step < max_steps:
    for x, y in train_loader:
        if step >= max_steps:
            break
            
        x, y = x.to(device), y.to(device)
        
        # Ensure divisible by patch_size
        valid_len = (x.size(1) // config.patch_size) * config.patch_size
        if valid_len < config.patch_size:
            continue
        x = x[:, :valid_len]
        y = y[:, :valid_len]
        
        # Update learning rate
        lr = get_lr(step)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        
        # Forward pass
        logits = model(x)
        loss = F.cross_entropy(logits.view(-1, 256), y.view(-1))
        loss = loss / gradient_accumulation_steps
        
        # Backward pass
        loss.backward()
        accum_loss += loss.item()
        
        # Gradient accumulation
        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            
            train_losses.append(accum_loss)
            accum_loss = 0
        
        # Evaluation
        if step % eval_interval == 0 and step > 0:
            val_loss = evaluate()
            val_losses.append(val_loss)
            pbar.set_postfix({
                'train_loss': f'{np.mean(train_losses[-100:]):.4f}',
                'val_loss': f'{val_loss:.4f}',
                'lr': f'{lr:.2e}'
            })
            
            # Save best model
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save({
                    'step': step,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'config': config.__dict__,
                    'val_loss': val_loss,
                }, 'checkpoints_tool_calling/best.pt')
                print(f'\nNew best model saved! Val loss: {val_loss:.4f}')
        
        # Periodic save
        if step % save_interval == 0 and step > 0:
            torch.save({
                'step': step,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'config': config.__dict__,
                'train_losses': train_losses,
                'val_losses': val_losses,
            }, f'checkpoints_tool_calling/step_{step}.pt')
        
        step += 1
        pbar.update(1)

pbar.close()
print('Training complete!')

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(train_losses, alpha=0.3)
ax.plot(np.convolve(train_losses, np.ones(100)/100, mode='valid'), label='Train (smoothed)')
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Training Loss')
ax.legend()

ax = axes[1]
ax.plot(np.arange(0, len(val_losses)) * eval_interval, val_losses, 'o-')
ax.set_xlabel('Step')
ax.set_ylabel('Loss')
ax.set_title('Validation Loss')

plt.tight_layout()
plt.savefig('tool_calling_training.png', dpi=150)
plt.show()

---
## Step 5: Test Tool Calling

In [ ]:
def generate_tool_call(model, tools, user_query, max_new=200, temperature=0.7):
    """Generate a tool call given tools and user query."""
    model.eval()
    
    # Format prompt
    prompt = f"<|system|>You are a helpful assistant with tool access.\n"
    prompt += f"<|tools|>{json.dumps(tools)}\n"
    prompt += f"<|user|>{user_query}\n"
    prompt += "<|assistant|>"
    
    # Tokenize (bytes)
    tokens = torch.tensor([[b for b in prompt.encode('utf-8')]], dtype=torch.long, device=device)
    
    # Ensure divisible by patch_size
    pad_len = (config.patch_size - tokens.size(1) % config.patch_size) % config.patch_size
    if pad_len > 0:
        tokens = F.pad(tokens, (0, pad_len), value=0)
    
    generated = []
    
    with torch.no_grad():
        for _ in range(max_new):
            # Get prediction
            logits = model(tokens[:, -1024:])  # Use last 1024 tokens
            logits = logits[:, -1, :] / temperature
            probs = F.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)
            
            tokens = torch.cat([tokens, next_token], dim=1)
            generated.append(next_token.item())
            
            # Stop at newline (end of tool call)
            if next_token.item() == ord('\n'):
                break
    
    # Decode
    response = bytes(generated).decode('utf-8', errors='replace').strip()
    
    # Try to parse as JSON
    try:
        tool_call = json.loads(response)
        return tool_call, True
    except:
        return response, False

In [ ]:
# Load best model
checkpoint = torch.load('checkpoints_tool_calling/best.pt', map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"Loaded best model (val_loss: {checkpoint['val_loss']:.4f})")

In [ ]:
# Test tool calling
test_tools = [
    {
        "name": "get_weather",
        "description": "Get current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            }
        }
    },
    {
        "name": "search_web",
        "description": "Search the web",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {"type": "string"}
            }
        }
    }
]

test_queries = [
    "What's the weather in San Francisco?",
    "Search for the best pizza places",
    "How's the temperature in London in celsius?",
    "Find information about machine learning",
]

print('='*60)
print('TOOL CALLING TEST')
print('='*60)

for query in test_queries:
    result, is_valid = generate_tool_call(model, test_tools, query)
    status = '✓' if is_valid else '✗'
    print(f'\nQuery: {query}')
    print(f'Response {status}: {result}')

---
## Step 6: Model Composition Test

Load a language model and compose with the tool-calling model.

In [ ]:
# Load a pre-trained language model (if available)
language_model_path = 'checkpoints_hierarchical_small/best.pt'

if os.path.exists(language_model_path):
    print('Loading language model for composition...')
    lang_checkpoint = torch.load(language_model_path, map_location=device)
    lang_config = HierarchicalBDHConfig(**lang_checkpoint.get('config', {}))
    lang_model = HierarchicalBDH(lang_config).to(device)
    lang_model.load_state_dict(lang_checkpoint['model_state_dict'])
    print('Language model loaded!')
else:
    print(f'No language model found at {language_model_path}')
    print('Train one first using train_hierarchical_wikitext2.ipynb')
    lang_model = None

In [ ]:
# Composed model that can do both language and tool calling
if lang_model is not None:
    import torch.nn as nn
    
    class ComposedToolAgent(nn.Module):
        """
        Compose language model with tool-calling model.
        
        Uses a simple routing mechanism:
        - If tools are present in prompt, weight tool model higher
        - Otherwise, use language model
        """
        
        def __init__(self, lang_model, tool_model):
            super().__init__()
            self.lang_model = lang_model
            self.tool_model = tool_model
            
            # Freeze both models
            for p in self.lang_model.parameters():
                p.requires_grad = False
            for p in self.tool_model.parameters():
                p.requires_grad = False
            
            # Learnable routing weights
            self.tool_weight = nn.Parameter(torch.tensor(0.5))
            
        def forward(self, x, has_tools=False):
            # Get logits from both models
            lang_logits = self.lang_model(x)
            tool_logits = self.tool_model(x)
            
            # Route based on context
            if has_tools:
                w = torch.sigmoid(self.tool_weight + 1)  # Bias toward tool model
            else:
                w = torch.sigmoid(self.tool_weight - 1)  # Bias toward lang model
            
            return w * tool_logits + (1 - w) * lang_logits
    
    composed_agent = ComposedToolAgent(lang_model, model).to(device)
    print('Composed agent created!')
    print(f'Total parameters: {sum(p.numel() for p in composed_agent.parameters()):,}')

In [ ]:
# Test composed agent
if lang_model is not None:
    print('='*60)
    print('COMPOSED AGENT TEST')
    print('='*60)
    
    # Test language capability
    print('\n--- Language Test ---')
    prompt = "The quick brown fox"
    tokens = torch.tensor([[b for b in prompt.encode('utf-8')]], device=device)
    
    # Pad
    pad_len = (32 - tokens.size(1) % 32) % 32
    tokens = F.pad(tokens, (0, pad_len), value=0)
    
    with torch.no_grad():
        # Generate with composed model
        for _ in range(50):
            logits = composed_agent(tokens, has_tools=False)
            next_token = logits[:, -1, :].argmax(dim=-1, keepdim=True)
            tokens = torch.cat([tokens, next_token], dim=1)
    
    output = bytes(tokens[0].cpu().tolist()).decode('utf-8', errors='replace')
    print(f'Prompt: {prompt}')
    print(f'Output: {output}')
    
    # Test tool calling capability
    print('\n--- Tool Calling Test ---')
    result, is_valid = generate_tool_call(model, test_tools, "What's the weather in Tokyo?")
    print(f'Tool call result: {result} (valid: {is_valid})')

---
## Summary

This notebook trains a specialized tool-calling BDH model that can:

1. Parse tool definitions in JSON format
2. Match user queries to appropriate tools
3. Generate valid JSON function calls

The model can be composed with a language model for a full agent.

In [ ]:
# Save final model for composition
torch.save({
    'model_state_dict': model.state_dict(),
    'config': config.__dict__,
    'model_type': 'tool_calling',
}, 'checkpoints_tool_calling/final_tool_model.pt')

print('Model saved for composition!')
print('\nTo compose with another model:')
print('1. Load this model')
print('2. Load a language model')
print('3. Use ComposedToolAgent class')
print('4. Fine-tune the routing weights on mixed data')